In [1]:
import numpy as np
import time as T
from joblib import Parallel, delayed
from scipy.interpolate import griddata,interpn,RegularGridInterpolator
import math
import os
import datetime
import cv2
from ftplib import FTP
import rioxarray
import xarray as xr
import paramiko
from scp import SCPClient
import subprocess
import datetime as dt

In [2]:
class H8_data:
    def __init__(self , account , pw , band , band_number , date):
        self.account = account
        self.pw = pw
        self.band = band
        self.band_number = band_number
        self.date = date
    
    def get_path(self):
        return '/data01/GEO/ORGDATA/H8AHI/hmwr829gr.cr.chiba-u.ac.jp/gridded/FD/V20151105/' + self.date[0:6] + '/' + self.band.upper() + '/'

    def get_filename(self):
        return self.date + "." + self.band + "." + self.band_number + ".fld.geoss.bz2"
    
    def DN2TBB(self,data):
        LUT=np.loadtxt(DN_PATH + 'count2tbb_v102/' + self.band + "." + self.band_number)
        return LUT[data,1]
    
    def file_path(self):
        return self.get_path() + self.get_filename() 
                 
#     def download_H8data_2015(self):
#         client = paramiko.SSHClient()
#         client.load_system_host_keys()
#         client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
#         client.connect(hostname='10.4.123.47', port=22, username=self.account, password=self.pw)
#         scp = SCPClient(client.get_transport())
#         sftp = client.open_sftp()

#         try :
#             sftp.stat(self.file_path())

#         except FileNotFoundError:
#             print("File Not Found")
#             return 'No data'

#         else:
#             scp.get(self.file_path(), folder_original_2015+'/')
#             p = subprocess.Popen('lbzip2 -d {}{}'.format(folder_original_2015+'/',self.file_path()[-33:]),shell=True)
#             p.communicate()
#             print ('Himawari8/AHI data Processed Finish')
#             return folder_original_2015 + '/' + self.get_filename()[:-4]
    
    def download_H8data_2015(self):
        ftp_dl_url = '/gridded/FD/V20151105/'  + self.date[0:6] + '/' + self.band.upper() + '/' + self.get_filename() 
        ftp = FTP()
        ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
        ftp.login()
        local_file = folder_original_2015 +'/' + self.get_filename() 
        with open(local_file, 'wb') as f:
            ftp.retrbinary('RETR ' + ftp_dl_url, f.write, 1024*1024)
        p = subprocess.Popen('lbzip2 -d {}'.format(local_file),shell=True)
        p.communicate()    
        ftp.close()
        return folder_original_2015 + '/' + self.get_filename()[:-4]
    
    def download_H8data_2019(self):
        ftp_dl_url = '/gridded/FD/V20190123/'  + self.date[0:6] + '/' + self.band.upper() + '/' + self.get_filename() 
        ftp = FTP()
        ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
        ftp.login()
        local_file = folder_original_2019 +'/' + self.get_filename() 
        with open(local_file, 'wb') as f:
            ftp.retrbinary('RETR ' + ftp_dl_url, f.write, 1024*1024)
        p = subprocess.Popen('lbzip2 -d {}'.format(local_file),shell=True)
        p.communicate()    
        ftp.close()
        return folder_original_2019 + '/' + self.get_filename()[:-4]
    
    
    def read_H8data_2019(self):
        H8_file_path = self.download_H8data_2019()
        
        if self.band == "vis":
            sr = 12000
        elif self.band == "ext":
            sr = 24000
        else:
            sr = 6000
        if H8_file_path != 'No data':
            with open(H8_file_path,'rb') as fp:
                data = np.frombuffer(fp.read(),dtype='>u2').reshape(sr,sr)
                data = self.DN2TBB(data)
                data = data/100

            print("data reading finish")
            return data
        else:
            return 'No data'
        
    def read_H8data_2015(self):
        H8_file_path = self.download_H8data_2015()
        
        if self.band == "vis":
            sr = 12000
        elif self.band == "ext":
            sr = 24000
        else:
            sr = 6000
        if H8_file_path != 'No data':
            with open(H8_file_path,'rb') as fp:
                data = np.frombuffer(fp.read(),dtype='>u2').reshape(sr,sr)
                data = self.DN2TBB(data)
                data = data/100

            print("data reading finish")
            return data
        else:
            return 'No data'   
        
        

In [3]:
DN_PATH = '/Users/Liwei/Downloads/'
folder_original_2019 = '/Users/Liwei/Downloads/2019'
AHI_2019 = H8_data('liwei','liwei00','ext','01','201601010000').read_H8data_2019()
folder_original_2015 = '/Users/Liwei/Downloads/2015'
AHI_2015 = H8_data('liwei','liwei00','ext','01','201601010000').read_H8data_2015()

data reading finish
data reading finish


In [8]:
abs(AHI_2019 - AHI_2015).max()

0.70235293